In [ ]:
# Transformer 라이브러리 (복습) - BERT 외 Transformer를 기반으로한 self-supervised learning method들 
# 학습. inference

# Transformers?

Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems. 2017

![screensh](https://blog.kakaocdn.net/dn/blla7d/btqBPXAzWdA/1yMKSf4SYWRT9t0yDt2lM1/img.jpg)

# huggingface library를 이용한 모델 불러오기

In [1]:
!pip install transformers[sentencepiece] 
# sentencepiece: 같이 깔리는 tokenizer는 default가 wordpiece이므로 따로 sentencepiece를 받기로 함 
!pip install datasets

     |████████████████████████████████| 3.4 MB 8.2 MB/s 
     |████████████████████████████████| 596 kB 73.1 MB/s 
     |████████████████████████████████| 3.3 MB 48.5 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 68.2 MB/s 
     |████████████████████████████████| 1.2 MB 56.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 8.0 MB/s 
     |████████████████████████████████| 1.1 MB 63.9 MB/s 
     |████████████████████████████████| 133 kB 79.6 MB/s 
     |████████████████████████████████| 243 kB 79.4 MB/s 
     |████████████████████████████████| 271 kB 76.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 144 kB 76.7 MB/s 


In [2]:
import torch
import torch.nn as nn
import torchtext

In [ ]:
# 모델구조: BERT, GPT, ... -> Transformer 
# - Encoder / Decoder / Encoder + Decoder 
# - layer가 몇 개인가, vector dimension이 몇인가 
# - pre-training 방법 (가장 중요함)
# - fine-tuning을 어느 데이터에 했는지 

# 총 26,663 개 모델 

In [3]:
# 26663개 모델 -> 한 줄만 바꿔서 해결 

# BERT 
from transformers import BertTokenizer, BertForTokenClassification
# bert인데, token classification용...  (필기하기에 너무 빠름...)

# token classification: 각 token마다 classification (0 or 1)

# huggingface.co의 models에 가면 'bert-base-uncased'가 있다. (model 종류)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Q. tokenizer는 왜 pretrained 되어야 할까? 
# A-1. 더 일반적인 토큰 
# - 사전 학습 데이터는 매우 큼. 더 일반적인 토큰들이 포함되어 있음
# A-2. model과 세트임. 
# - word embedding할 때, 
#   <pad> -> 0번, it -> 32번
#   <pad> -> 0번, it -> 25번 
#   25번째 embedding vector / 32번째 embedding vector 는 서로 다름 기존에 학습한 것 그대로 사용해야 함
# -> 새로운 단어를 추가하고 싶으면? 
#  - 기존 30000개 vocab, -> 30001번부터 사용

model = BertForTokenClassification.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
model

In [19]:
inputs = tokenizer.encode(["Hello, my dog is cute"], return_tensors="pt")
# tensor='pt': tensor pytorch / 'tf': tensorflow / (default): list
outputs = model(inputs)[0]

In [22]:
# Electra
# GAN 형태 (discriminator)

from transformers import ElectraTokenizer, ElectraModel
# logits: softmax 취하기 전 값
# for token classification -> classification 모델 
# BertModel + Linear layer (어떤 output을 쓸 것인가)
# 한 토큰마다 1개의 classification 결과 

# BertFerSentenceClassification
# BertModel + Linear layer 
# 한 문장에 1개의 classification 결과

# liklihood: softmax 취한 후 값 

tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model = ElectraModel.from_pretrained('google/electra-large-discriminator')

inputs = tokenizer.encode("The capital of France is [MASK].", return_tensors="pt")

outputs = model(inputs)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/668 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
outputs

BaseModelOutputWithPastAndCrossAttentions([('last_hidden_state',
                                            tensor([[[-0.1712,  0.0506,  0.6153,  ...,  0.3136,  0.0033, -0.1737],
                                                     [ 0.2136,  0.2114,  0.0852,  ...,  0.0938, -0.4353, -0.0061],
                                                     [-0.0450,  0.1075,  0.2639,  ..., -0.0869, -0.2249,  0.2051],
                                                     ...,
                                                     [-0.1608,  0.1830,  0.5355,  ...,  0.1005, -0.0964, -0.2321],
                                                     [ 0.2983,  0.0679,  0.1625,  ..., -0.0437, -0.1851,  0.4285],
                                                     [-0.1704,  0.1914,  0.5484,  ...,  0.1241, -0.0846, -0.2425]]],
                                                   grad_fn=<NativeLayerNormBackward0>))])

In [24]:
outputs.last_hidden_state

tensor([[[-0.1712,  0.0506,  0.6153,  ...,  0.3136,  0.0033, -0.1737],
         [ 0.2136,  0.2114,  0.0852,  ...,  0.0938, -0.4353, -0.0061],
         [-0.0450,  0.1075,  0.2639,  ..., -0.0869, -0.2249,  0.2051],
         ...,
         [-0.1608,  0.1830,  0.5355,  ...,  0.1005, -0.0964, -0.2321],
         [ 0.2983,  0.0679,  0.1625,  ..., -0.0437, -0.1851,  0.4285],
         [-0.1704,  0.1914,  0.5484,  ...,  0.1241, -0.0846, -0.2425]]],
       grad_fn=<NativeLayerNormBackward0>)

In [25]:
outputs.last_hidden_state.shape

torch.Size([1, 9, 1024])

# AutoModel 사용해보기

In [26]:
from transformers import AutoModelForTokenClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased')

inputs = tokenizer.encode("Hello, my dog is cute", return_tensors="pt")
outputs = model(inputs)[0]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

# 사전학습된 모델 사용해보기

In [27]:
# pre-traing
# fine_tuning
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Q&A 데이터셋
# SQuAD v2 -> 질문을 하면 생성해서 대답(서술형) -> 생성모델
# SQuAD v1 -> 질문을 하면 지문에서 어디에 있는지 맞춤(객관식) 

# T5: encoding-decoding 구조 
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-squadv2")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-squadv2")


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [30]:

def get_answer(question, context):
  input_text = "question: %s  context: %s" % (question, context) # 질문, 지문
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])
  # 생성 모델의 경우 model.generate
  # teacher forcing 
  
  return tokenizer.decode(output[0]) # answer 


# wikipeida text (비욘세에 대한 지문)
context = 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy'
question = 'When did Beyonce start becoming popular?'

get_answer(question,context)

'<pad> late 1990s</s>'

In [31]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:787: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [32]:
# 지문, 질문 -> 답(OA)
# 지문, 답 -> 질문(OG)
def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer([input_text], return_tensors='pt')

  output = model.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer.decode(output[0])

context = 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy'
answer = '1981'

get_question(answer, context)

'<pad> question: What year was Beyonce born?</s>'

# huggingfcae 라이브러리를 이용한 데이터 처리하기

In [33]:
from datasets import load_dataset
datasets = load_dataset('imdb') # 영화 리뷰

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
datasets.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [35]:
datasets['train'][0] # 0 or 1 negative / positive

{'label': 0,
 'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are f

In [36]:
# unsupervised VS supervised 
# 정답지(label) 유무
# self-supervised: 정답지를 스스로 만들어서 학습(본인이 문제내고 본인이 학습)

datasets['unsupervised'][0] 
# label: -1은 unsupervised

{'label': -1,
 'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)'}

In [37]:
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        x = item['text']
        y = item['label']
        return x, torch.tensor(y).long()


In [38]:
train_dataset = CustomDataset(datasets['train'])
test_dataset = CustomDataset(datasets['test'])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [39]:
batch = next(iter(train_dataloader))

In [40]:
features = tokenizer(list(batch[0]))

Token indices sequence length is longer than the specified maximum sequence length for this model (586 > 512). Running this sequence through the model will result in indexing errors


In [42]:
batch

[("Claire Denis has demonstrated repeatedly that film does not need to tell a story, that it is sufficient to create an experience that allows the viewer to take the ingredients and make of them what they will.<br /><br />Ostensibly the idea within the framework of a most non-linear film is the older man living on the French-Swiss border, a man devoted to his dogs, who still has a lover, but whose cardiac status increasingly threatens his life. He has a son with a little family who infrequently meet with him, but when he discovers he is in need of a heart transplant he opts for going to Tahiti via Japan to obtain a heart transplant on the black market and to rekindle a long lost relationship with a son he had form a Tahitian women years ago.<br /><br />What Denis does with this outline of a story is use her camera to explore the loneliness of the soul, the vastness of nature, man's interaction with people vs animals, etc. Much of the time the 'film' doesn't make sense, but that is beca

In [41]:
tokenizer(list(batch[0])).keys()

# 앞문장, 뒷문장 구분 X -> BERT 때는 있었는데 지금은 없는 이유? 
# -> T5 모델은 문장 두 개를 구분하는 NSP를 안함 

dict_keys(['input_ids', 'attention_mask'])

In [43]:
features.keys()

dict_keys(['input_ids', 'attention_mask'])

# 짧은 코드만으로 학습을 시켜봅시다.

In [44]:
from datasets import load_dataset
datasets = load_dataset('imdb')

from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        x = item['text']
        y = item['label']
        return x, torch.tensor(y).long()


train_dataset = CustomDataset(datasets['train'])
test_dataset = CustomDataset(datasets['test'])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

# cuda
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(DEVICE)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 아래는 필요없음. 모델에 내장되어 있으므로... 
# criterion = nn.CrossEntropyLoss()

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [49]:
def train_epoch(model, dataloader, tokenizer, optimizer):
    model.train()
    train_loss = 0
    for i, (x,y) in enumerate(dataloader):
        # x: text
        # y: 정답 label
        features = tokenizer(list(x), padding='max_length', return_tensors='pt',max_length=512, truncation=True)
        # padding 512
        # 600 -> 512 (truncation)
        x = features['input_ids'].to(DEVICE)
        attention_mask = features['attention_mask'].to(DEVICE)
        y = y.to(DEVICE)
        loss = model(x, labels=y)['loss']

        # model update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        if i % 50 == 0:
            print('Iter [{}/{}] Loss {:.6f}'.format(i+1, len(dataloader), train_loss / (i+1)))
    
    return train_loss / len(dataloader)

def test_epoch(model, dataloader, tokenizer):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
      for x,y in dataloader:
          x = tokenizer(list(x), padding='max_length', return_tensors='pt',max_length=512)['input_ids'].to(DEVICE)
          out = model(x)['logits']
          pred = out.argmax(-1)
          preds.append(pred.cpu())
          labels.append(y)
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    acc = (preds == labels).float().mean()
    print('ACC : {:.3f}'.format(acc))
    return preds, labels

def predict(model, tokenizer, sentence):
    model.eval()
    x = tokenizer.encode(sentence, return_tensors='pt').to(DEVICE)
    out = model(x)['logits']
    pred = out.argmax(-1)
    return pred.cpu()

In [50]:
EPOCHS=1

for i in range(EPOCHS):
    train_epoch(model, train_dataloader, tokenizer, optimizer)
    test_epoch(model, test_dataloader, tokenizer)

Iter [1/6250] Loss 1.130178
Iter [51/6250] Loss 0.709343
Iter [101/6250] Loss 0.703329
Iter [151/6250] Loss 0.707499
Iter [201/6250] Loss 0.712182
Iter [251/6250] Loss 0.712198
Iter [301/6250] Loss 0.710737
Iter [351/6250] Loss 0.708935
Iter [401/6250] Loss 0.709833
Iter [451/6250] Loss 0.709804
Iter [501/6250] Loss 0.708593
Iter [551/6250] Loss 0.708734
Iter [601/6250] Loss 0.709943
Iter [651/6250] Loss 0.709727
Iter [701/6250] Loss 0.709752
Iter [751/6250] Loss 0.710150
Iter [801/6250] Loss 0.709308
Iter [851/6250] Loss 0.709722
Iter [901/6250] Loss 0.710428
Iter [951/6250] Loss 0.710154
Iter [1001/6250] Loss 0.710766
Iter [1051/6250] Loss 0.711671
Iter [1101/6250] Loss 0.712048
Iter [1151/6250] Loss 0.711507
Iter [1201/6250] Loss 0.711231
Iter [1251/6250] Loss 0.710616
Iter [1301/6250] Loss 0.710450
Iter [1351/6250] Loss 0.710037


KeyboardInterrupt: ignored

# 과제

- Text summary 에 fine-tuned 되어있는 모델을 불러와 아래의 글들을 요약해봅시다.
- 정상적으로 보이는 글이 완성되면 과제 통과입니다. 
- 완벽하게 요약하지 않아도 됩니다. 완전 이상한 글만 아니면 통과입니다!
    - 이상한 글 예시: 이 글은 이 이 글은, . , , pad 이 것  (학습이 제대로 안 된 결과)
    - 정상적인 글 예시: 이건 과제에 관한 글이다

In [95]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-summarization")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-summarization")

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [100]:
text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠를 즐긴다."

# 1. tokenizer를 이용해 토크나이즈를 진행합니다. (huggingface library에 있는 예제를 참고해보세요.)
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

# 2. model.generate 함수를 이용해 생성해봅시다.
summary_ids = model.generate(torch.tensor([input_ids]))

# 3. tokenizer 를 이용해 decode하여 읽을 수 있는 글로 바꿔줍니다.
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)


'TV가 없는 집도 많고, TV가 없는 집도 많아진 만큼 미디어의 혜택을'

In [101]:
text = '수학에서 순환소수인 0.999…는 실수 1의 또 다른 십진법 소수 표현이다. 즉 "0.999…"와 "1"은 같은 수이다. 이러한 증명은 실수론의 전개, 배경이 있는 가정, 역사적 맥락, 대상이 되는 청자(듣는 사람) 등에 맞는 수준에 따른 것으로서 여러 단계의 수학적 엄밀함을 적절하게 고려한 다양한 정식화가 있다.'

# 위의 코드를 가져와 반복해보세요.
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'수학에서 순환소수인 0.999는 실수 1의 또 다른 십진법'

In [102]:
text = '암모니아(영어: ammonia)는 질소와 수소로 이루어진 화합물이다. 분자식은 NH3이다. 상온에서는 특유의 자극적인 냄새가 나는 무색의 기체 상태로 존재하고있다. 대기 중에도 소량의 양이 포함되어 있으며, 천연수에 미량 함유되어 있기도 하다. 토양 중에도 세균의 질소 유기물의 분해 과정에서 생겨난 암모니아가 존재할 수 있다. 대표적인 반자성체 중 하나이다.'

# 위의 코드를 가져와 반복해보세요.
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'토양 중에도 질소와 수소로 이루어진 화합물인 암모니아가 존재할'

## 직접 학습 모델 만들어보기

In [104]:
# model 고르기 (예시) 

import torch
import torch.nn as nn
class FreezeModel(nn.Module):
    def __init__(self,):
        from transformers import AutoModel
        model_name = 't5-small'
        # hidden state vector
        self.t5 = AutoModel.from_pretrained(model_name)

        # for param_name, parameter in self.t5.named_parameters():
        #     parameter.requires_grad = False

        self.out_layer = nn.Linear(512, 30000) # 512 dim vector -> 30000개 vocab

    def forward(self, x):
        with torch.no_grad():
            x = self.t5(x) # gradient 계산 X
        
        x = self.out_layer(x)
        return x

### 1. data 받기 (task 정하기)
### 2. model, tokenizer 고르기 + optimizer 고르기 (+ criterion 고르기)
### 3. dataset, dataloader 코딩 
### 4. 학습 코드 짜기 
### 5. 평가 코드 짜기 

In [53]:
# 1. data 받아오기
dataset_name = 'xsum'
datasets = load_dataset(dataset_name)

# 2. model 골라서 tokenizer, model 불러오기, optimizer 만들기
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.cuda() # gpu

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# 3. custom dataset class
from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        # documnet: 입력 데이터
        # summary: 출력 데이터(정답지)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        x = item['document'] # text
        y = item['summary'] # text
        return x, y

train_dataset = CustomDataset(datasets['train'])
valid_dataset = CustomDataset(datasets['validation'])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=False) # validation
# shuffle -> 랜덤하게 섞는 것 
# 평가, 테스트를 할 때는 섞을 필요 없다! 


Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [63]:
# 4. 학습 코드, 진행

def tokenizing(batch):
    document = batch[0]
    summary = batch[1]
    document_features = tokenizer(list(document), return_tensors='pt',
                                  padding='max_length', max_length=512, truncation=True)
    summary_features = tokenizer(list(summary), return_tensors='pt', 
                                padding='max_length', max_length=512, truncation=True)
    return document_features, summary_features 

for epoch in range(5):
    model.train()
    train_loss = 0

    for batch in train_dataloader:
        # tokenizing + tensor + gpu upload
        document_features, summary_features = tokenizing(batch)
        loss = model(document_features['input_ids'].to(DEVICE), # gpu 
                    attention_mask = document_features['attention_mask'].to(DEVICE),
                    labels = summary_features['input_ids'].to(DEVICE))['loss']

        optimizer.zero_grad() # 누적 방지(초기화)
        loss.backward() 
        optimizer.step() # update

        train_loss += loss.item() # tensor.item(): [1] -> 1

        if i % 100 == 0: 
            print('iter[{}/{}] train loss [{:.6f}]'.format(i, 
                                                           len(train_dataloader), 
                                                           train_loss / (i+1)))

    # 1 epoch 끝나면 출력
    print('train loss: {:.5f}'.format(train_loss/len(train_dataloader)))

# truncation: 길이가 너무 길면 512로 잘라줌
# padding: 길이가 짧으면 512로 늘려줌 

# 5. 평가 진행

# validation
model.eval()
valid_loss = 0
with torch.no_grad():
    # gradient 그래프 유지 X -> 속도 4~5배 정도 빠름. 웬만하면 넣어주자
    for batch in valid_dataloader:
        # tokenizing + tensor + gpu upload
        document_features, summary_features = tokenizing(batch)
        loss = model(document_features['input_ids'].to(DEVICE), # gpu 
                    attention_mask = document_features['attention_mask'].to(DEVICE),
                    labels = summary_features['input_ids'].to(DEVICE))['loss']

        ## 학습 때 사용하는 코드이므로 평가때는 넣지 않는다! 
        # optimizer.zero_grad() # 누적 방지(초기화)
        # loss.backward() 
        # optimizer.step() # update

        valid_loss += loss.item() # tensor.item(): [1] -> 1

print('valid loss: {:.5f}'.format(valid_loss/len(valid_dataloader)))

# 너무 오래 걸리니 돌리진 말자.... 

iter[0/51012] train loss [0.170589]
iter[0/51012] train loss [0.331838]
iter[0/51012] train loss [0.509369]
iter[0/51012] train loss [0.690929]
iter[0/51012] train loss [0.861997]
iter[0/51012] train loss [0.998835]
iter[0/51012] train loss [1.184670]
iter[0/51012] train loss [1.303534]
iter[0/51012] train loss [1.461806]
iter[0/51012] train loss [1.573346]
iter[0/51012] train loss [1.746851]
iter[0/51012] train loss [1.945674]
iter[0/51012] train loss [2.023106]
iter[0/51012] train loss [2.172205]
iter[0/51012] train loss [2.353001]
iter[0/51012] train loss [2.499704]
iter[0/51012] train loss [2.687311]
iter[0/51012] train loss [2.830500]
iter[0/51012] train loss [2.987862]
iter[0/51012] train loss [3.139602]
iter[0/51012] train loss [3.291566]
iter[0/51012] train loss [3.440620]
iter[0/51012] train loss [3.631608]
iter[0/51012] train loss [3.764483]
iter[0/51012] train loss [3.903070]
iter[0/51012] train loss [4.066982]
iter[0/51012] train loss [4.261197]
iter[0/51012] train loss [4.

KeyboardInterrupt: ignored

In [73]:
model.generate(tokenizer.encode(datasets['validation'][0]['document'], return_tensors='pt').cuda())

tensor([[    0,    71,   388,    65,   118,  4977,    28,  7712,    11,  7712,
             6,    28,   192, 10740,    11,   192, 10740,    16,  2135,    28]],
       device='cuda:0')

In [ ]:
# freeze -> word embedding (CBOW, Skip-Gram)
# BERT, ELECTRA (contextual, representation) -> freeze 해도 됨. 전부 다 학습하되, 
# 기존에 사전학습된 부분은 lr 작게, 새로합습하는 부분은 lr 크게

# 데이터셋 크기가 작을수록 freeze 쪽이 성능이 높고, 
# 데이터셋 크기가 클수록 전부 학습하는 쪽이 성능이 높다. 
# NLP -> 대부분 데이터가 크다;; 

In [ ]:
# length penalty: 짧을수록 penalty

# beam search 
# word1 word2 w3 w4 ... P(word1) * P(word2) * P(w3) * P(w4) .... = P(sentence1)
# 긴문장은 만들기 어렵고, 짧은 문장은 쉬움
# P(word1) * P*(word2) End.